In [2]:
import sys 
sys.path.append('..')
from utils.paths import make_dirs, make_dir, POTATOES_LIST, POTATOES_CLEANED_DIR, JSON_DIR_ASSETS, HASHTAGS_PER_USER_DIR, HASHTAG_FREQUENCY_DIR_ALL, CLEANED_DIR, HASHTAG_FREQUENCY_DIR, GREEDY_MODULARITY_SELECTED_DIR, HASHTAG_TOP_DIR, POSTS_DIR_JSON, CLUSTERS_DIR_JSON, HASHTAG_FREQUENCY_DIR_JSON
from utils.utils import load_dataframes, load_dataframe, write_dict_to_csv, write_list_to_csv, write_df_to_csv, read_result_csv
import pandas as pd
import os
from glob import glob
import json
import numpy as np
import math

make_dirs()

## CSV to JSON
- convert processed data to the format required by the interactive web application

In [8]:
def csv_to_json(fromDir, toDir, quarter):
    df = pd.read_csv(os.path.join(fromDir, quarter + ".csv"), index_col=0)
    result = df.to_json()

    with open(os.path.join(toDir, quarter + '.json'), 'w') as f:
        f.write(result)

### Hashtag frequency per district

In [9]:
def frequencies_to_json():
    for potato in POTATOES_LIST():
        csv_to_json(HASHTAG_FREQUENCY_DIR, HASHTAG_FREQUENCY_DIR_JSON, potato)

frequencies_to_json()

177
177
12
12
124
124
14
14
77
77
69
69
334
334
193
193
1408
1408
35
35
28
28
10
10
14
14
8
8
151
151
57
57
66
66
103
103
173
173
43
43
2691
2691
111
111
1159
1159
338
338
39
39
8
8
55
55
859
859
22
22
326
326
14
14
7
7
2
2
481
481
843
843
9
9
55
55
67
67
3304
3304
11
11
97
97
3
3
623
623
41
41
213
213
1186
1186
281
281
4783
4783
48
48
240
240
131
131
174
174
593
593
50
50
409
409
4032
4032
1468
1468
484
484
37
37
447
447
494
494
53
53
20
20
797
797
32
32
3
3


### Clusters and uniqueness per district

In [2]:
min_count = 10
res = '1.8'
max_n_hashtags_per_cluster = 15

def checkRow(x, hashtags):
    if x in hashtags:
        return True
    else:
        return False

path = os.path.join(GREEDY_MODULARITY_SELECTED_DIR, 'min_count_' + str(min_count) + '_res_' + res)

for quarter in POTATOES_LIST():
    path_quarter_clusters = os.path.join(path, quarter)
    files_quarter = sorted(glob(path_quarter_clusters + '/*_.csv'))
    df = load_dataframe(HASHTAG_TOP_DIR, quarter)
    df = df.drop(df['count_quarter'].idxmax())

    df['hashtag'] = df['hashtag'].astype(str)
    df['cluster'] = ""
    
    for file in files_quarter:
        df_cluster = pd.read_csv(file)
        hashtags = list(df_cluster['hashtag'])
        if len(hashtags) > 2:
            hashtags = hashtags[:max_n_hashtags_per_cluster]
            name = hashtags[0]
            mask = (df['hashtag'].apply(lambda x: checkRow(x, hashtags)))
            df['cluster'][mask] = name

    top_count = np.max(df['count_quarter'])
    df['count'] = df['count_quarter']
    df['uniqueness'] = np.round(df['uniqueness'], decimals=2)
    df['rank'] = np.round(df['rank'], decimals=2)
    df['radius'] = df['count']/math.pi
    df['radius'] = np.sqrt(df['radius'])
    df['radius'] = df['radius']/df['radius'].max()
    df = df[['hashtag', 'count', 'radius', 'uniqueness', 'cluster', 'rank']]

    out_path = os.path.join(CLUSTERS_DIR_JSON, quarter + '.json')
    df.to_json(out_path, orient='records')

>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark


/var/folders/6c/_dzq3mkj1pn46mx5cn0zmsg00000gn/T/ipykernel_21956/2875232821.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'][mask] = name


>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now loading: maxvorstadt
>> now loading: messestadtriem
>> now loading: milbertshofen
>> now loading: mittersendling
>> now loading: 

### Posts per district

In [12]:
def posts_to_json():
    dir = POTATOES_CLEANED_DIR
    for quarter in POTATOES_LIST():
        path = os.path.join(dir, quarter + ".csv")
        df = pd.read_csv(path, index_col=0) 
        df.reset_index(drop=True, inplace=True)
        if (df['is_video'].dtype != bool) :
            df.replace({'is_video': {'True': True, 'False': False}}, inplace=True)

        df['is_video'] = df['is_video'].astype(bool)
        df = df[~df['is_video']]
        df = df.sort_values(by='likes', ascending=False)

        df.reset_index(drop=True, inplace=True)
        df = df[['post_url', 'hashtags', 'likes']]        
        result = df.to_json()

        with open(os.path.join(POSTS_DIR_JSON, quarter + '.json'), 'w') as f:
            f.write(result)

posts_to_json()

../../data/cleaned/potatoes/posts/allach.csv
../../data/cleaned/potatoes/posts/altperlach.csv
../../data/cleaned/potatoes/posts/altstadt.csv
../../data/cleaned/potatoes/posts/amhart.csv
../../data/cleaned/potatoes/posts/arabellapark.csv
../../data/cleaned/potatoes/posts/au.csv
../../data/cleaned/potatoes/posts/aubing.csv
../../data/cleaned/potatoes/posts/bergamlaim.csv
../../data/cleaned/potatoes/posts/bogenhausen.csv
../../data/cleaned/potatoes/posts/borstei.csv
../../data/cleaned/potatoes/posts/daglfing.csv
../../data/cleaned/potatoes/posts/denning.csv
../../data/cleaned/potatoes/posts/fasanerie.csv
../../data/cleaned/potatoes/posts/fasangarten.csv
../../data/cleaned/potatoes/posts/feldmoching.csv
../../data/cleaned/potatoes/posts/forstenried.csv
../../data/cleaned/potatoes/posts/freiham.csv
../../data/cleaned/potatoes/posts/freimann.csv
../../data/cleaned/potatoes/posts/fröttmaning.csv
../../data/cleaned/potatoes/posts/fürstenried.csv
../../data/cleaned/potatoes/posts/giesing.csv
..

### Posts whole city

In [14]:
min_likes = 100

def posts_to_json_city():
    dir = POTATOES_CLEANED_DIR

    df_all = pd.DataFrame(columns=['post_url', 'hashtags', 'likes', 'is_video'])
    for quarter in POTATOES_LIST():
        path = os.path.join(dir, quarter + ".csv")
        df = pd.read_csv(path, index_col=0) 
        df.reset_index(drop=True, inplace=True)
        if (df['is_video'].dtype != bool) :
            df.replace({'is_video': {'True': True, 'False': False}}, inplace=True)

        df['is_video'] = df['is_video'].astype(bool)
        df = df[~df['is_video']]
        df_all = df_all.append(df)

    df = df.drop_duplicates(subset='post_url', keep="first")

    df_all['likes'] = df_all['likes'].astype(int)
    df_all = df_all.sort_values(by='likes', ascending=False)
    df_all.reset_index(drop=True, inplace=True)
    df_all.drop(df_all[df_all['hashtags'] == '[]'].index, inplace = True)
    print(len(df_all))
    df_all.drop(df_all[df_all['likes'] < min_likes].index, inplace = True)
    print(len(df_all))

    df_all = df_all[['post_url', 'hashtags', 'likes']]    
    print(df_all[:10])    
    result = df_all.to_json()
    new_dir = os.path.join(dir, 'json')
    make_dir(new_dir)

    with open(os.path.join(new_dir, 'city.json'), 'w') as f:
        f.write(result)

posts_to_json_city()

../../data/cleaned/potatoes/posts/allach.csv
../../data/cleaned/potatoes/posts/altperlach.csv
../../data/cleaned/potatoes/posts/altstadt.csv
../../data/cleaned/potatoes/posts/amhart.csv
../../data/cleaned/potatoes/posts/arabellapark.csv
../../data/cleaned/potatoes/posts/au.csv
../../data/cleaned/potatoes/posts/aubing.csv
../../data/cleaned/potatoes/posts/bergamlaim.csv
../../data/cleaned/potatoes/posts/bogenhausen.csv
../../data/cleaned/potatoes/posts/borstei.csv
../../data/cleaned/potatoes/posts/daglfing.csv
../../data/cleaned/potatoes/posts/denning.csv
../../data/cleaned/potatoes/posts/fasanerie.csv
../../data/cleaned/potatoes/posts/fasangarten.csv
../../data/cleaned/potatoes/posts/feldmoching.csv
../../data/cleaned/potatoes/posts/forstenried.csv
../../data/cleaned/potatoes/posts/freiham.csv
../../data/cleaned/potatoes/posts/freimann.csv
../../data/cleaned/potatoes/posts/fröttmaning.csv
../../data/cleaned/potatoes/posts/fürstenried.csv
../../data/cleaned/potatoes/posts/giesing.csv
..

### Stats per district

In [4]:
def potato_list_count_users_stories():

    df_post_count = load_dataframe(CLEANED_DIR, 'post_counts')
    min_count = 10
    res = '1.8'

    df_post_count['stories'] = 0
    df_post_count['users'] = 0

    path_to_clusters = os.path.join(GREEDY_MODULARITY_SELECTED_DIR, 'min_count_' + str(min_count) + '_res_' + res)

    for quarter in POTATOES_LIST():
        path_quarter_clusters = os.path.join(path_to_clusters, quarter)
        files_quarter = sorted(glob(path_quarter_clusters + '/*_.csv'))

        stories_count = len(files_quarter)
        df_users = load_dataframe(HASHTAGS_PER_USER_DIR, quarter)
    
        users_count = len(df_users.hashtags)
        print(users_count)

        index = df_post_count.loc[df_post_count['quarter'] == quarter].index

        df_post_count['stories'][index] = stories_count
        df_post_count['users'][index] = users_count
        
        

    df_post_count['post_count'] = df_post_count['post_count_cleaned']
    print(df_post_count)

    df_post_count = df_post_count[['quarter','post_count','users', 'stories']]
    df_post_count.to_json(os.path.join(JSON_DIR_ASSETS, 'posts_users_stories.json'), orient='records')

potato_list_count_users_stories()

>> now loading: post_counts
>> now loading: allach
1134
>> now loading: altperlach
160
>> now loading: altstadt
1101
>> now loading: amhart
173
>> now loading: arabellapark
678
>> now loading: au


/var/folders/6c/_dzq3mkj1pn46mx5cn0zmsg00000gn/T/ipykernel_22091/3179633333.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_post_count['stories'][index] = stories_count
/var/folders/6c/_dzq3mkj1pn46mx5cn0zmsg00000gn/T/ipykernel_22091/3179633333.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_post_count['users'][index] = users_count


1264
>> now loading: aubing
1232
>> now loading: bergamlaim
1076
>> now loading: bogenhausen
3891
>> now loading: borstei
450
>> now loading: daglfing
398
>> now loading: denning
91
>> now loading: fasanerie
237
>> now loading: fasangarten
119
>> now loading: feldmoching
1013
>> now loading: forstenried
439
>> now loading: freiham
600
>> now loading: freimann
964
>> now loading: fröttmaning
1091
>> now loading: fürstenried
312
>> now loading: giesing
7397
>> now loading: hadern
782
>> now loading: haidhausen
3081
>> now loading: harlaching
1174
>> now loading: harras
566
>> now loading: harthof
174
>> now loading: hasenbergl
593
>> now loading: hellabrunn
5006
>> now loading: herzogpark
206
>> now loading: isarvorstadt
1460
>> now loading: johanneskirchen
242
>> now loading: kieferngarten
111
>> now loading: kirchtrudering
20
>> now loading: laim
1772
>> now loading: lehel
2922
>> now loading: lerchenau
178
>> now loading: lochhausen
427
>> now loading: ludwigsvorstadt
476
>> now loadi

## Top hashtags city

In [ ]:

def checkRow(x, hashtags):
    if x in hashtags:
        return True
    else:
        return False

def count_all_quarters_to_json():
    df = load_dataframe(HASHTAG_FREQUENCY_DIR_ALL, 'count_all_quarters')
    print(len(df))
    df.drop(df[df['count'] <100].index, inplace = True)
    print(len(df))

    potatoes = POTATOES_LIST()

    df.drop(df[df['hashtag'].apply(lambda x: checkRow(x, potatoes))].index, inplace = True)
    print(len(df))
    
    
    df = df[['hashtag']]
    # df = df.set_index('hashtag')
    
    print(df)

    df.to_json(os.path.join(JSON_DIR_ASSETS, 'top_hashtags_city.json'))

    df = df.sort_values(by='hashtag')
    df.to_json(os.path.join(JSON_DIR_ASSETS, 'top_hashtags_city_sorted.json'))

In [ ]:
# df = load_dataframe(POTATOES_DIR, 'post_count')
# # df = df[['hashtag', 'count', 'opacitysigmoidlog']]
# # highest_rank = max(df['rank'])
# # df['rank'] = df['rank'] / highest_rank


# # df.columns= ['text', 'count', 'opacity']
# # df['opacity'] = np.round(df['opacity'], decimals=2)

# # path = os.path.join(HASHTAG_TOP_DIR_JSON, quarter + '.json')
# df.set_index('quarter', inplace=True)

# test_dict = df.to_dict(orient='index')
# print(test_dict)

# test_dict2 = {}

# for k in test_dict:
#     test_dict2[k] = test_dict[k]['post_count']

# print(test_dict2)


# with open(POST_COUNT_DIR_JSON + '/post_count.json', "w") as outfile:
#     json.dump(test_dict2, outfile)

### Convert posts to JSON

In [ ]:

# def checkRow(rowHashtags, hashtags):
#     if len(set.intersection(set(rowHashtags), set(hashtags))) > 0:
#         return True
#     else:
#         return False
    
#     # df = dfs[quarter]
# # df = load_dataframe(HASHTAG_TOP_DIR, 'westend')
# # print(', '.join(list(df['hashtag'])))


# # df = QUARTER_RESULTS('westend')
# # df_new = df[df['hashtag'].apply()]

# # hashtags = SEARCH_HASHTAGS_LIST()
# selected_columns = ['owner_id','shortcode','hashtags', 'caption', 'post_url']

# df = QUARTER_RESULTS('westend')
# df = df[selected_columns]
# print(len(df))
# mask = (df['hashtags'].apply(lambda x: checkRow(x, hashtags)))
# # print(mask)

# # print(df[:10])
# df = df[mask]
# print(len(df))
# print(df[:10])
